In [ ]:
import numpy as np
import tensorflow as tf
import keras
import pandas as pd
import cv2
from glob import glob
import matplotlib.pyplot as plt
from pathlib import Path
from keras.layers import Input, Lambda, Dense, Flatten, Conv2D, Activation, MaxPool2D, Dropout
from keras.models import  Model
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential
from keras.preprocessing import image, image_dataset_from_directory

ImportError: ignored

In [ ]:
train_path = "/content/drive/MyDrive/covid_dataset/Training"
test_path= "/content/drive/MyDrive/covid_dataset/Testing"


In [ ]:
# use the image data generator to import the image from the dataset
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
train_datagen = ImageDataGenerator(rescale = 1/255,
                                   shear_range= 0.2,
                                   zoom_range= 0.2,
                                   horizontal_flip= True)
test_datagen = ImageDataGenerator(rescale = 1/255)
training_set = train_datagen.flow_from_directory("/content/drive/MyDrive/covid_dataset/Training",
                                                 target_size=(224, 224),
                                                 color_mode='rgb',
                                                 class_mode='categorical',
                                                 batch_size= 30)
test_set = test_datagen.flow_from_directory("/content/drive/MyDrive/covid_dataset/Testing",
                                               target_size=(224, 224),
                                                 color_mode='rgb',
                                                 class_mode='categorical',
                                                 batch_size= 20)

Found 2534 images belonging to 2 classes.
Found 560 images belonging to 2 classes.


In [ ]:
training_set.class_indices

{'Covid19': 0, 'Normal': 1}

In [ ]:
#create CNN model 
model1= Sequential(Conv2D(64, kernel_size=(3,3), input_shape=(224,224,3)))
model1.add(Activation("relu"))
model1.add(MaxPool2D(pool_size=(2,2)))
model1.add(Conv2D(64, kernel_size=(3,3)))
model1.add(Activation("relu"))
model1.add(MaxPool2D(pool_size=(2,2)))

model1.add(Conv2D(128, kernel_size=(3,3)))
model1.add(Activation("relu"))
model1.add(MaxPool2D(pool_size=(2,2)))

model1.add(Conv2D(64, kernel_size=(3,3)))
model1.add(Activation("relu"))
model1.add(MaxPool2D(pool_size=(2,2)))

model1.add(Flatten())
model1.add(Dense(64, activation="relu"))
model1.add(Dropout(0.5))
model1.add(Dense(32, activation="relu"))
model1.add(Dropout(0.5))
model1.add(Dense(2, input_dim= 128,activation="softmax"))


In [ ]:
model1.compile(
    optimizer='adam', 
    loss= "categorical_crossentropy",
     loss_weights=None, 
      run_eagerly=None,
       steps_per_execution=None,
      metrics= ["accuracy"])

In [ ]:
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 64)      1792      
_________________________________________________________________
activation (Activation)      (None, 222, 222, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 64)      36928     
_________________________________________________________________
activation_1 (Activation)    (None, 109, 109, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 128)       7

In [ ]:
###Fit the model

In [ ]:
vggmodel= model1.fit_generator( training_set, 
                              steps_per_epoch= 6,
                              epochs=20,  
                              validation_data= test_set)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
6/6 [==============================] - 202s 30s/step - loss: 0.7022 - accuracy: 0.5647 - val_loss: 0.7606 - val_accuracy: 0.5000
Epoch 2/20
6/6 [==============================] - 42s 7s/step - loss: 0.7482 - accuracy: 0.5585 - val_loss: 0.7123 - val_accuracy: 0.5000
Epoch 3/20
6/6 [==============================] - 38s 6s/step - loss: 0.7074 - accuracy: 0.5298 - val_loss: 0.6954 - val_accuracy: 0.5000
Epoch 4/20
2/6 [=========>....................] - ETA: 20s - loss: 0.6889 - accuracy: 0.5083

In [ ]:
#add preprocessing layer to the front of VGG
Image_size = [224,224]
vgg= VGG19(include_top=False, 
           weights='imagenet',
           input_tensor=None,
           input_shape= Image_size + [3],
           pooling=None, 
           classes=2, 
           classifier_activation= "softmax")

#don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

#useful for getting number of classes
#folders= glob("covid19/train/*") 


In [ ]:
#our layers- we can adjust number of layer as per our requirement
x = Flatten()(vgg.output)
#prediction = Dense(len(folders), activation= "softmax")(x)
prediction = Dense(2, activation= "softmax")(x)

In [ ]:
#create a model object
model = Model(inputs= vgg.input, outputs = prediction)

In [ ]:


#cost and optimization initialization for model
model.compile(
    optimizer='adam', 
    loss= "categorical_crossentropy",
     loss_weights=None, 
      run_eagerly=None,
       steps_per_execution=None,
      metrics= ["accuracy"])

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
#fit VGG19 model 
vggmodel= model.fit_generator( training_set, 
                              steps_per_epoch= 6,
                              epochs=20,  
                              validation_data= test_set)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
6/6 [==============================] - 6s 1s/step - loss: 0.6958 - accuracy: 0.5222 - val_loss: 0.7071 - val_accuracy: 0.5000
Epoch 2/20
6/6 [==============================] - 6s 946ms/step - loss: 0.6818 - accuracy: 0.5732 - val_loss: 0.6975 - val_accuracy: 0.5000
Epoch 3/20
6/6 [==============================] - 6s 1s/step - loss: 0.6877 - accuracy: 0.5444 - val_loss: 0.7006 - val_accuracy: 0.5000
Epoch 4/20
6/6 [==============================] - 5s 937ms/step - loss: 0.6737 - accuracy: 0.5944 - val_loss: 0.7114 - val_accuracy: 0.5000
Epoch 5/20
6/6 [==============================] - 5s 974ms/step - loss: 0.6792 - accuracy: 0.5667 - val_loss: 0.7516 - val_accuracy: 0.5000
Epoch 6/20
6/6 [==============================] - 5s 935ms/step - loss: 0.7980 - accuracy: 0.5333 - val_loss: 0.7026 - val_accuracy: 0.5000
Epoch 7/20
6/6 [==============================] - 6s 1s/step - loss: 0.6922 - accuracy: 0.5389 - val_loss: 0.6940 - val_accuracy: 0.5000
Epoch 8/20
6/6 [=============